In [26]:
import json 
import numpy as np 
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

In [27]:
with open('intents.json') as file:
    data = json.load(file)
    
training_sentences = []
training_labels = []
labels = []
responses = []


for intent in data['intents']:
    for pattern in intent['patterns']:
        training_sentences.append(pattern)
        training_labels.append(intent['tag'])
    responses.append(intent['responses'])
    
    if intent['tag'] not in labels:
        labels.append(intent['tag'])
        
num_classes = len(labels)

In [28]:
lbl_encoder = LabelEncoder()
lbl_encoder.fit(training_labels)
training_labels = lbl_encoder.transform(training_labels)

In [29]:
vocab_size = 1000
embedding_dim = 16
max_len = 20
oov_token = "<OOV>"

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded_sequences = pad_sequences(sequences, truncating='post', maxlen=max_len)

In [30]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(GlobalAveragePooling1D())
model.add(Dense(16, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', 
              optimizer='adam', metrics=['accuracy'])

model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 20, 16)            16000     
                                                                 
 global_average_pooling1d_3   (None, 16)               0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dense_9 (Dense)             (None, 16)                272       
                                                                 
 dense_10 (Dense)            (None, 16)                272       
                                                                 
 dense_11 (Dense)            (None, 9)                 153       
                                                                 
Total params: 16,697
Trainable params: 16,697
Non-trainable params: 0
__________________________________________________

In [31]:

epochs = 500
history = model.fit(padded_sequences, np.array(training_labels), epochs=epochs)

Epoch 1/500
2/2 [==============================] - 0s 6ms/step - loss: 2.1973 - accuracy: 0.0263
Epoch 2/500
2/2 [==============================] - 0s 8ms/step - loss: 2.1959 - accuracy: 0.1579
Epoch 3/500
2/2 [==============================] - 0s 8ms/step - loss: 2.1949 - accuracy: 0.2105
Epoch 4/500
2/2 [==============================] - 0s 6ms/step - loss: 2.1943 - accuracy: 0.2895
Epoch 5/500
2/2 [==============================] - 0s 8ms/step - loss: 2.1938 - accuracy: 0.3158
Epoch 6/500
2/2 [==============================] - 0s 7ms/step - loss: 2.1934 - accuracy: 0.2368
Epoch 7/500
2/2 [==============================] - 0s 7ms/step - loss: 2.1927 - accuracy: 0.1579
Epoch 8/500
2/2 [==============================] - 0s 7ms/step - loss: 2.1923 - accuracy: 0.1579
Epoch 9/500
2/2 [==============================] - 0s 5ms/step - loss: 2.1915 - accuracy: 0.1579
Epoch 10/500
2/2 [==============================] - 0s 7ms/step - loss: 2.1906 - accuracy: 0.1579
Epoch 11/500
2/2 [===========

In [32]:
# to save the trained model
model.save("chat_model")

import pickle

# to save the fitted tokenizer
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
# to save the fitted label encoder
with open('label_encoder.pickle', 'wb') as ecn_file:
    pickle.dump(lbl_encoder, ecn_file, protocol=pickle.HIGHEST_PROTOCOL)

INFO:tensorflow:Assets written to: chat_model/assets


In [33]:
import json 
import numpy as np
from tensorflow import keras
from sklearn.preprocessing import LabelEncoder

import colorama 
colorama.init()
from colorama import Fore, Style, Back

import random
import pickle

with open("intents.json") as file:
    data = json.load(file)


def chat():
    # load trained model
    model = keras.models.load_model('chat_model')

    # load tokenizer object
    with open('tokenizer.pickle', 'rb') as handle:
        tokenizer = pickle.load(handle)

    # load label encoder object
    with open('label_encoder.pickle', 'rb') as enc:
        lbl_encoder = pickle.load(enc)

    # parameters
    max_len = 20
    
    while True:
        print(Fore.LIGHTBLUE_EX + "User: " + Style.RESET_ALL, end="")
        inp = input()
        if inp.lower() == "quit":
            break

        result = model.predict(keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences([inp]),
                                             truncating='post', maxlen=max_len))
        tag = lbl_encoder.inverse_transform([np.argmax(result)])

        for i in data['intents']:
            if i['tag'] == tag:
                print(Fore.GREEN + "ChatBot:" + Style.RESET_ALL , np.random.choice(i['responses']))

        # print(Fore.GREEN + "ChatBot:" + Style.RESET_ALL,random.choice(responses))

print(Fore.YELLOW + "Start messaging with the bot (type quit to stop)!" + Style.RESET_ALL)
chat()

Start messaging with the bot (type quit to stop)!
1/1 [==============================] - 0s 71ms/step
ChatBot: Hello
1/1 [==============================] - 0s 32ms/step
ChatBot: Have a nice day
1/1 [==============================] - 0s 25ms/step
ChatBot:  Listen to this articleThe move has long been planned as part of an additional change for 2024 where tyre blanket use is set to be totally abandoned within the championships sustainability drive. Because this is the period of blanket that is using a lot more energy  its like the oven at home he continued. And to propose a full 2023 strategy of tyre blanket heating two hours at 70C. Thats the plan for the moment.It is understood that Pirelli has also presented its 2023 tyre blanket idea to the FIA and the F1 organisation and has received positive feedback to the new approach rather than lowering the blanket temperature to 50C. Isola also suggested that the inevent tyre testing conducted in Austin and Mexico could return at certain event